## Dataset

#### Este conjunto de datos analiza la relación entre la salud mental y diversos factores demográficos, de estilo de vida y relacionados con el trabajo. Contiene información sobre género, edad, presión laboral, satisfacción en el trabajo, duración del sueño, hábitos alimenticios, estrés financiero, horas de trabajo y varios indicadores de salud mental, como depresión, pensamientos suicidas e historial familiar de enfermedades mentales.

##### El objetivo del dataset es ofrecer una comprensión más profunda de cómo las condiciones de vida y laborales afectan el bienestar mental, lo que lo hace ideal para análisis exploratorios, modelado predictivo e investigaciones estadísticas.



## Librerias utilizadas

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import numpy as np

### Leer dataset desde un archivo CSV


In [12]:
dataset = pd.read_csv(rf'C:\Users\ashli\OneDrive\Escritorio\LaSalle\7TO\MineríaDatos\3P\Depression_Professional_Dataset.csv')
dataset.head()

,Gender,Age,Work Pressure,Job Satisfaction,Sleep Duration,Dietary Habits,Have you ever had suicidal thoughts ?,Work Hours,Financial Stress,Family History of Mental Illness,Depression
0,Female,37,2.0,4.0,7-8 hours,Moderate,No,6,2,No,No
1,Male,60,4.0,3.0,5-6 hours,Unhealthy,Yes,0,4,Yes,No
2,Female,42,2.0,3.0,5-6 hours,Moderate,No,0,2,No,No
3,Female,44,3.0,5.0,7-8 hours,Healthy,Yes,1,2,Yes,No
4,Male,48,4.0,3.0,7-8 hours,Moderate,Yes,6,5,Yes,No


#### Este archivo cuenta con un total de 2054 registros y un total de 11 columnas

In [13]:
# Obtener el número de registros (filas) y columnas
num_registros, num_columnas = dataset.shape

# Imprimir los resultados
print(f"El dataset tiene {num_registros} registros y {num_columnas} columnas.")

El dataset tiene 2054 registros y 11 columnas.


## Limpieza de datos

#### Manejo de valores faltantes

In [14]:

# Identificar valores faltantes
print("Valores faltantes por columna:")
print(dataset.isnull().sum())

# Opción 1: Rellenar valores faltantes (imputación)
dataset['Age'].fillna(dataset['Age'].median(), inplace=True)  # Ejemplo con la mediana
dataset['Sleep Duration'].fillna('Unknown', inplace=True)    # Rellenar texto categórico
dataset['Job Satisfaction'].fillna(dataset['Job Satisfaction'].mean(), inplace=True)



# Verificar nuevamente
print("Valores faltantes después de la limpieza:")
print(dataset.isnull().sum())

Valores faltantes por columna:
Gender                                   0
Age                                      0
Work Pressure                            0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Have you ever had suicidal thoughts ?    0
Work Hours                               0
Financial Stress                         0
Family History of Mental Illness         0
Depression                               0
dtype: int64
Valores faltantes después de la limpieza:
Gender                                   0
Age                                      0
Work Pressure                            0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Have you ever had suicidal thoughts ?    0
Work Hours                               0
Financial Stress                         0
Family History of Mental Illness         0
Depression 

C:\Users\ashli\AppData\Local\Temp\ipykernel_20200\1096970811.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['Age'].fillna(dataset['Age'].median(), inplace=True)  # Ejemplo con la mediana
C:\Users\ashli\AppData\Local\Temp\ipykernel_20200\1096970811.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

#### Eliminacion de valores duplicados

In [15]:
# Verificar duplicados
duplicados = dataset.duplicated().sum()
print(f"Cantidad de filas duplicadas: {duplicados}")

# Eliminar duplicados
dataset = dataset.drop_duplicates()

# Verificar si quedan duplicados
print(f"Cantidad de filas después de eliminar duplicados: {dataset.duplicated().sum()}")


Cantidad de filas duplicadas: 0
Cantidad de filas después de eliminar duplicados: 0


#### Verificacion de inconsistencias 

In [16]:
# Verificar valores únicos en columnas categóricas para encontrar inconsistencias
print("Valores únicos en 'Gender':", dataset['Gender'].unique())
print("Valores únicos en 'Sleep Duration':", dataset['Sleep Duration'].unique())

# Estandarizar texto en columnas categóricas
dataset['Gender'] = dataset['Gender'].str.strip().str.capitalize()  # Limpiar espacios y uniformar formato
dataset['Sleep Duration'] = dataset['Sleep Duration'].replace({
    '7-8 hours': '7-8 Hours',
    '5-6 hours': '5-6 Hours',
    'Unknown': 'Unknown'
})

# Verificar nuevamente los valores únicos
print("Valores únicos corregidos en 'Gender':", dataset['Gender'].unique())
print("Valores únicos corregidos en 'Sleep Duration':", dataset['Sleep Duration'].unique())


Valores únicos en 'Gender': ['Female' 'Male']
Valores únicos en 'Sleep Duration': ['7-8 hours' '5-6 hours' 'More than 8 hours' 'Less than 5 hours']
Valores únicos corregidos en 'Gender': ['Female' 'Male']
Valores únicos corregidos en 'Sleep Duration': ['7-8 Hours' '5-6 Hours' 'More than 8 hours' 'Less than 5 hours']


### Preparar entorno para guardar en dataset en una base de datos MySQL

In [17]:
!pip install pymysql
!pip install mysql-connector-python


   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ------------------------------------ --- 41.0/45.0 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 739.0 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/16.1 MB 1.6 MB/s eta 0:00:10
    --------------------------------------- 0.2/16.1 MB 2.1 MB/s eta 0:00:08
   - -------------------------------------- 0.4/16.1 MB 3.4 MB/s eta 0:00:05
   -- ------------------------------------- 0.9/16.1 MB 5.2 MB/s eta 0:00:03
   -- ------------------------------------- 1.2/16.1 MB 5.8 MB/s eta 0:00:03
   ---- ----------------------------------- 1.6/16.1 MB 5.7 MB/s eta 0:00:03
   ---- ----------------------------------- 1.8/16.1 MB 6.1 MB/s eta 0:00:03
   ------ --------------------------------- 2.6/16.1 MB 7.3 MB/s eta 0:00:02
   ------- -------------------------------- 3.1/16.1 MB 7.6 MB/s eta 0:00:02
   -------- ------------------------------- 3.5/16.1 MB 7.8 MB/s eta 0:00:02
   ---------- ----------------------------- 4.1/16.1 MB 8.3 MB/s eta 0:00:02
   ----------- ---------------------------- 4.6/16.1 MB 8.3 MB/s eta 0:00:02
   ---


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Preparar conexion a MySQL desde Python

In [18]:
import pymysql 

# Configuración de conexión
conn = pymysql.connect(
    host='localhost',           # Dirección del servidor MySQL 
    user='root',          
    password='JJLovesU1!',  
    database='Depression_Professional', 
    port=3306                   # Puerto (por defecto, 3306)
)

cursor = conn.cursor()
print("Conexión exitosa a la base de datos")


RuntimeError: 'cryptography' package is required for sha256_password or caching_sha2_password auth methods

### Creacion de la tabla en MySQL para el dataset

In [ ]:
# Crear una tabla desde Python 
create_table_query = """
CREATE TABLE IF NOT EXISTS depression_data (
    Gender VARCHAR(10),
    Age INT,
    Work_Pressure FLOAT,
    Job_Satisfaction FLOAT,
    Sleep_Duration VARCHAR(50), -- Por si hay textos más largos
    Dietary_Habits VARCHAR(50), -- Igual que arriba
    Suicidal_Thoughts VARCHAR(10), -- Para mayor flexibilidad
    Work_Hours INT,
    Financial_Stress FLOAT,
    Family_History VARCHAR(10), -- Para valores como "Yes/No" con margen
    Depression VARCHAR(10) -- Igualmente flexible
);

"""
cursor.execute(create_table_query)
print("Tabla creada (o ya existía).")


Tabla creada (o ya existía).


In [ ]:
print(dataset.columns)
print(dataset.shape)  # Número de filas y columnas


Index(['Gender', 'Age', 'Work Pressure', 'Job Satisfaction', 'Sleep Duration',
       'Dietary Habits', 'Have you ever had suicidal thoughts ?', 'Work Hours',
       'Financial Stress', 'Family History of Mental Illness', 'Depression'],
      dtype='object')
(2054, 11)


### Insercion de los datos del dataset en la tabla

In [ ]:
# Renombrar las columnas para coincidir con los nombres en la base de datos
dataset = dataset.rename(columns={
    'Have you ever had suicidal thoughts ?': 'Suicidal_Thoughts',
    'Family History of Mental Illness': 'Family_History'
})

# Convertir el DataFrame a una lista de tuplas
data_tuples = [tuple(row) for row in dataset.to_numpy()]

# Query de inserción en la base de datos
insert_query = """
INSERT INTO depression_data (Gender, Age, Work_Pressure, Job_Satisfaction, Sleep_Duration, 
                              Dietary_Habits, Suicidal_Thoughts, Work_Hours, 
                              Financial_Stress, Family_History, Depression)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Insertar los datos en lotes
try:
    cursor.executemany(insert_query, data_tuples)
    conn.commit()
    print("Datos insertados exitosamente.")
except pymysql.MySQLError as e:
    print(f"Error al insertar datos: {e}")

# Cerrar la conexión
cursor.close()
conn.close()


Datos insertados exitosamente.
